# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/Users/mackd/OneDrive/Desktop/Python_API_Challenge/WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,13.15,79,10,6.73,SH,1730342357
1,1,uturoa,-16.7333,-151.4333,25.42,71,70,7.27,PF,1730342358
2,2,afaahiti,-17.7500,-149.2833,23.04,70,49,7.29,PF,1730342359
3,3,west bay,19.3667,-81.4167,28.04,78,40,3.60,KY,1730342360
4,4,tralee,52.2704,-9.7026,12.17,89,96,1.05,IE,1730342361


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
humidity_map= city_data_df.hvplot.points(
    'Lng','Lat',
    geo=True,
    tiles= 'OSM',
    frame_width= 1100, frame_height= 900,
    size= 'Humidity',
    scale=1,
    color= 'City'
)
#Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
MyCity = city_data_df[
    (city_data_df['Humidity'] < 62) & 
    (city_data_df['Max Temp'] > 24)]
# Drop any rows with null values
MyCity_cleaned=city_data_df.dropna()

# Display sample data
MyCity_cleaned

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,13.15,79,10,6.73,SH,1730342357
1,1,uturoa,-16.7333,-151.4333,25.42,71,70,7.27,PF,1730342358
2,2,afaahiti,-17.7500,-149.2833,23.04,70,49,7.29,PF,1730342359
3,3,west bay,19.3667,-81.4167,28.04,78,40,3.60,KY,1730342360
4,4,tralee,52.2704,-9.7026,12.17,89,96,1.05,IE,1730342361
...,...,...,...,...,...,...,...,...,...,...
573,573,togur,58.3631,82.8265,-3.14,88,98,1.86,RU,1730421184
574,574,keflavik,64.0049,-22.5624,1.24,51,0,6.69,IS,1730420842
575,575,kapa'a,22.0752,-159.3190,28.18,75,40,8.23,US,1730421187
576,576,huancavelica,-12.7667,-74.9833,6.33,67,60,1.32,PE,1730421188


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = MyCity_cleaned.copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,edinburgh of the seven seas,-37.0676,-12.3116,13.15,79,10,6.73,SH,1730342357,
1,1,uturoa,-16.7333,-151.4333,25.42,71,70,7.27,PF,1730342358,
2,2,afaahiti,-17.7500,-149.2833,23.04,70,49,7.29,PF,1730342359,
3,3,west bay,19.3667,-81.4167,28.04,78,40,3.60,KY,1730342360,
4,4,tralee,52.2704,-9.7026,12.17,89,96,1.05,IE,1730342361,
...,...,...,...,...,...,...,...,...,...,...,...
573,573,togur,58.3631,82.8265,-3.14,88,98,1.86,RU,1730421184,
574,574,keflavik,64.0049,-22.5624,1.24,51,0,6.69,IS,1730420842,
575,575,kapa'a,22.0752,-159.3190,28.18,75,40,8.23,US,1730421187,
576,576,huancavelica,-12.7667,-74.9833,6.33,67,60,1.32,PE,1730421188,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
  'categories': 'accommodation.hotel',
    'limit': 1,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat}{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
afaahiti - nearest hotel: No hotel found
west bay - nearest hotel: No hotel found
tralee - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
bodaybo - nearest hotel: No hotel found
nichinan - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
thinadhoo - nearest hotel: No hotel found
nadi - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
prince albert - nearest hotel: No hotel found
toba tek singh -

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,edinburgh of the seven seas,-37.0676,-12.3116,13.15,79,10,6.73,SH,1730342357,No hotel found
1,1,uturoa,-16.7333,-151.4333,25.42,71,70,7.27,PF,1730342358,No hotel found
2,2,afaahiti,-17.7500,-149.2833,23.04,70,49,7.29,PF,1730342359,No hotel found
3,3,west bay,19.3667,-81.4167,28.04,78,40,3.60,KY,1730342360,No hotel found
4,4,tralee,52.2704,-9.7026,12.17,89,96,1.05,IE,1730342361,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
573,573,togur,58.3631,82.8265,-3.14,88,98,1.86,RU,1730421184,No hotel found
574,574,keflavik,64.0049,-22.5624,1.24,51,0,6.69,IS,1730420842,No hotel found
575,575,kapa'a,22.0752,-159.3190,28.18,75,40,8.23,US,1730421187,No hotel found
576,576,huancavelica,-12.7667,-74.9833,6.33,67,60,1.32,PE,1730421188,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
hotel_map=hotel_df.hvplot.points('Lng','Lat',geo = True, tiles = 'OSM', size='Humidity', color = 'City',width=600, height=300,hover_cols=['Lng','Lat','City','Humidity','Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)